### Traning code for the first version of Roberta, done in Google Colab.

In [ ]:
!pip install transformers datasets torch

import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Now you can use pandas to load the data from your drive
train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Y2BlC/Transformer model/reduced_on_translated.csv')
test_df = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Y2BlC/Transformer model/group 16_url1.xlsx')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-2-a294159db41e>:5: DtypeWarning: Columns (8,32,33,34,36) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Y2BlC/Transformer model/reduced_on_translated.csv')


In [ ]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

def preprocess_function(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=512)

train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)
train_dataset = train_dataset.map(preprocess_function, batched=True)
test_dataset = test_dataset.map(preprocess_function, batched=True)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/31722 [00:00<?, ? examples/s]

Map:   0%|          | 0/2466 [00:00<?, ? examples/s]

In [ ]:
def format_dataset(dataset):
    dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
    return dataset

train_dataset = format_dataset(train_dataset)
test_dataset = format_dataset(test_dataset)


In [ ]:
model_path = '/content/drive/My Drive/Path/To/Save/Model'
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)


In [ ]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification, TrainingArguments, Trainer
from datasets import Dataset
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score

from google.colab import drive
drive.mount('/content/drive')

# Load your data
train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Y2BlC/Transformer model/comb_cropped_df_2.csv')
test_df = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Y2BlC/Transformer model/predictions_group16testset.xlsx')

# Encode labels
label_encoder = LabelEncoder()
train_df['labels'] = label_encoder.fit_transform(train_df['main_category'])
test_df['labels'] = label_encoder.transform(test_df['labels'])

# Initialize tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

# Function to tokenize the data
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True, max_length=128)

# Convert to Dataset objects
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Load RoBERTa model
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=len(label_encoder.classes_))

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    metric_for_best_model="f1",
    logging_dir='./logs',
    evaluation_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    save_total_limit=2  # Keep only the 2 most recent model checkpoints
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

# Train the model
trainer.train()

# Optionally, save the model and tokenizer for later use
model_path = "/content/drive/MyDrive/Colab Notebooks/Y2BlC/Trained Models/best_roberta"
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Map:   0%|          | 0/49987 [00:00<?, ? examples/s]

Map:   0%|          | 0/2466 [00:00<?, ? examples/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.707800,1.794863,0.437145,0.430435
2,0.561000,1.705001,0.503244,0.483794
3,0.424300,2.138684,0.465126,0.457372
4,0.375800,2.852526,0.465531,0.456472


('/content/drive/MyDrive/Colab Notebooks/Y2BlC/Trained Models/best_roberta/tokenizer_config.json',
 '/content/drive/MyDrive/Colab Notebooks/Y2BlC/Trained Models/best_roberta/special_tokens_map.json',
 '/content/drive/MyDrive/Colab Notebooks/Y2BlC/Trained Models/best_roberta/vocab.json',
 '/content/drive/MyDrive/Colab Notebooks/Y2BlC/Trained Models/best_roberta/merges.txt',
 '/content/drive/MyDrive/Colab Notebooks/Y2BlC/Trained Models/best_roberta/added_tokens.json')